https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
import sys
sys.path.append('../ames') # path the the directory
import config
from data_prep import clean, add_features, dummify

In [3]:
housing = clean(config.HOUSING_CSV)
housing = add_features(housing)
housing = pd.read_pickle(config.HOUSING_PICKLE)

In [21]:
the_chosen_variables = ['1stFlrSF','2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 
             'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 
             'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath', 
             'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 
             'HalfBath', 'KitchenAbvGr', 'LotArea', 'LotFrontage', 
             'LowQualFinSF', 'MasVnrArea', 'MiscVal', 'MoSold', 'MSSubClass', 
             'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 
             'ScreenPorch', 'TotalBsmtSF', 'TotRmsAbvGrd', 'WoodDeckSF', 
             'YearBuilt', 'YearRemodAdd', 'YrSold','DecadeBuilt', 'DecadeRemodel',
             'TotalLivingArea','UnusedLotSize','HasPool','HouseAge', 'HouseAgeSq',
             'Toilets','Showers']

In [22]:
the_chosen_variables.remove('GrLivArea')
the_chosen_variables.remove('1stFlrSF')
the_chosen_variables.remove('TotalBsmtSF')
the_chosen_variables.remove('BsmtFinSF1')
the_chosen_variables.remove('BsmtUnfSF')
the_chosen_variables.remove('LotArea')

In [23]:
limited_variables = ['TotalLivingArea','UnusedLotSize','HasPool','OverallQual','GarageArea','YearBuilt']
X = housing[the_chosen_variables]

dummy_vars = {'Neighborhood':'Nbhd','LotConfig':'LC','SaleCondition':'SC'}
dummy_df = dummify(housing, dummy_vars)
X = pd.concat([X,dummy_df], axis=1)
y = housing['SalePrice']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [26]:
from lightgbm import LGBMRegressor, cv
lgb = LGBMRegressor(random_state=42)
lgb.fit(X_train,y_train)
print(lgb.score(X_train, y_train))
print(lgb.score(X_test,y_test))

0.9776394030355313
0.9078138967973748


In [27]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(y_test, lgb.predict(X_test), squared=False)
print(rms)

23714.723395789384


## Feature Importance

In [28]:
d = pd.DataFrame(lgb.feature_importances_,index=X.columns,columns=['Importance'])
d.sort_values('Importance',ascending=False).head(25)

,Importance
TotalLivingArea,408
UnusedLotSize,294
GarageArea,230
MasVnrArea,172
2ndFlrSF,152
YearRemodAdd,148
LotFrontage,147
YearBuilt,133
OverallQual,129
OpenPorchSF,122
